# Training Autoencoders

## Experimenting autoencoders for denoising

In [1]:
#Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import keras

Using TensorFlow backend.


In [2]:
#Import Keras module
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, advanced_activations
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [3]:
#pretty plots
%matplotlib inline

#Load the index data
raw_data = pd.read_csv('price_only.csv', skiprows=1, parse_dates=['Date']).set_index(['Date'])
raw_data.tail()

,P,P_L1,P_L2,P_L3,P_L4,P_L5,P_L6,P_L7,P_L8,P_L9,...,P_L11,P_L12,P_L13,P_L14,P_L15,P_L16,P_L17,P_L18,P_L19,P_L20
Date,,,,,,,,,,,,,,,,,,,,,
2017-09-21,29.00,29.00,28.95,29.00,28.70,28.70,28.75,28.85,28.85,28.55,...,28.45,28.60,28.60,28.75,28.75,28.85,28.50,28.65,28.55,28.20
2017-09-22,28.75,29.00,29.00,28.95,29.00,28.70,28.70,28.75,28.85,28.85,...,28.40,28.45,28.60,28.60,28.75,28.75,28.85,28.50,28.65,28.55
2017-09-25,28.45,28.75,29.00,29.00,28.95,29.00,28.70,28.70,28.75,28.85,...,28.55,28.40,28.45,28.60,28.60,28.75,28.75,28.85,28.50,28.65
2017-09-26,28.50,28.45,28.75,29.00,29.00,28.95,29.00,28.70,28.70,28.75,...,28.85,28.55,28.40,28.45,28.60,28.60,28.75,28.75,28.85,28.50
2017-09-27,28.60,28.50,28.45,28.75,29.00,29.00,28.95,29.00,28.70,28.70,...,28.85,28.85,28.55,28.40,28.45,28.60,28.60,28.75,28.75,28.85


In [4]:
#Data and Labels split
y = raw_data['P']
X = raw_data

In [7]:
#Training, Validation, test split
train_size = 0.9
timestep = 5

In [9]:
#Calcualte cutoff
train_cut_index = int(train_size * X.shape[0] - train_size * X.shape[0] % timestep)
print(train_cut_index)


2235


In [10]:
#Split train, validation
X_train, X_val = X.iloc[0:train_cut_index,:], X.iloc[train_cut_index:,:]
y_train, y_val = y[0:train_cut_index], y[train_cut_index:]